# 02 - Mejoras: Overfitting, Dropout y EarlyStopping (MNIST)

En este notebook vamos a mejorar el entrenamiento del modelo:

- Veremos qué es **overfitting** (sobreajuste)
- Usaremos **Dropout** para regularizar
- Usaremos **EarlyStopping** para detener el entrenamiento en el mejor punto
- Graficaremos el historial (`history`) para entender qué está pasando

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## 1) Cargar y preparar datos (MNIST)

MNIST ya viene integrado en TensorFlow:
- Imágenes: 28x28 en escala de grises
- Labels: 0 a 9

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train.shape, y_train.shape

x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

## 2) ¿Qué es overfitting?

**Overfitting (sobreajuste)** sucede cuando:
- El modelo se vuelve *muy bueno* en los datos de entrenamiento
- Pero empeora (o no mejora) en datos nuevos (validación/test)

Se detecta cuando:
- `accuracy` (train) sube
- pero `val_accuracy` se estanca o baja
- y `val_loss` empieza a subir

Vamos a entrenar "más de la cuenta" para observarlo.

## 3) Modelo un poco más grande

Aumentaremos la capacidad del modelo (más neuronas), lo cual puede:
- mejorar precisión
- pero también facilitar overfitting


In [ ]:
x_train_small = x_train[:10000]
y_train_small = y_train[:10000]

model_overfit = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

model_overfit.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model_overfit.summary()

history_overfit = model_overfit.fit(
    x_train_small, y_train_small,
    epochs=30,
    batch_size=128,
    validation_split=0.2,
    verbose=1
)

## 5) Graficar historial (train vs validation)

Vamos a ver:
- `loss` vs `val_loss`
- `accuracy` vs `val_accuracy`

Esto nos ayuda a detectar overfitting visualmente.

In [ ]:
def plot_history(history, title="Training History"):
    hist = history.history
    
    plt.figure(figsize=(10, 4))
    plt.plot(hist["loss"], label="loss (train)")
    plt.plot(hist["val_loss"], label="loss (val)")
    plt.title(title + " - Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 4))
    plt.plot(hist["accuracy"], label="accuracy (train)")
    plt.plot(hist["val_accuracy"], label="accuracy (val)")
    plt.title(title + " - Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

plot_history(history_overfit, "Model WITHOUT Dropout")

## 5) Solución 1: Dropout (regularización)

**Dropout** apaga aleatoriamente un % de neuronas durante entrenamiento.

¿Por qué ayuda?
- Evita que el modelo dependa demasiado de un conjunto pequeño de neuronas
- Reduce overfitting
- Fuerza al modelo a aprender patrones más generales

Nota: Dropout se usa solo en entrenamiento, no en inferencia.

## 6) Solución 2: EarlyStopping

**EarlyStopping** detiene el entrenamiento si la validación ya no mejora.

Ventaja:
- Evita entrenar de más
- Nos quedamos con el mejor modelo automáticamente

Parámetros clave:
- `monitor`: qué observar (ej. val_loss)
- `patience`: cuántas épocas esperar sin mejora
- `restore_best_weights`: regresar al mejor punto


In [ ]:
model_regularized = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation="softmax")
])

model_regularized.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model_regularized.summary()

# EarlyStopping Callbacks
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

history_regularized = model_regularized.fit(
    x_train_small, y_train_small,
    epochs=30,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

plot_history(history_regularized, "Model WITH Dropout + EarlyStopping")


## 7) Evaluación final en test

Ahora medimos con datos que el modelo nunca vio.


In [ ]:
test_loss, test_acc = model_dropout.evaluate(x_test, y_test, verbose=0)
print("Test accuracy:", test_acc)
print("Test loss:", test_loss)